# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [27]:
import pandas as pd
import matplotlib as mpl
import sklearn as skl
import numpy as np
from sklearn.linear_model import LinearRegression

#Are squirrels who climb more often more likely to approach humans? (linear regression?)
#Which location has squirrels that are most likely to approach humans? (clustering , N,S,E,W)

data = pd.read_csv("2018_Central_Park_Squirrel_Census_-_Squirrel_Data_20250418.csv")
print("Hello World!")


Hello World!


In [47]:
##Does elevation impact the secondary fur color of squirrels? (Logistic regression) (predict squirle 2nd hair color)

In [50]:
#Climbing Squirrles (linear regression. Climb amount (# of times observed climbing) vs chance to approach humans)

#x = # of climb
#y = #of approach climbing

x = data[["Climbing"]]
y = data[["Approaches"]] #bools

#group = data.groupby("Squirrel Number")
#print(group)

#chance to appraoch humans :   approach human ammount / total squirrle ammount
#climb ammount = total squirrel observed vs climb amount

# Aggregate data by number of climbing observations
# Group by Climbing, count how many squirrels approached
grouped = data.groupby('Climbing')['Approaches'].agg(['sum', 'count']).reset_index()
grouped.columns = ['Climbing', 'NumApproaches', 'Total']

print(grouped)
# Calculate the "chance to approach" as a proportion
#grouped['ChanceToApproach'] = grouped['NumApproaches'] / grouped['Total']

# Use 'Total' (number observed climbing or not) as the independent variable
#x = grouped[['Total']]  # must be 2D for sklearn
#y = grouped['ChanceToApproach']

print(x)
print(y)
#for i in range(len(data)):

#print(x)
#print(y)

#model = LinearRegression()
#model.fit(x, y)

# Print coefficients
#print("Coefficient:", model.coef_[0])
#print("Intercept:", model.intercept_)

model = LinearRegression().fit(x,y)

coef = round(float(model.coef_), 2)
intercept = round(float(model.intercept_), 2)

print("Linear equation: y=", coef, "x+", intercept)

#predict = 43



   Climbing  NumApproaches  Total
0     False            151   2365
1      True             27    658
      Climbing
0        False
1        False
2        False
3        False
4        False
...        ...
3018     False
3019     False
3020     False
3021     False
3022     False

[3023 rows x 1 columns]
      Approaches
0          False
1          False
2          False
3          False
4          False
...          ...
3018       False
3019       False
3020       False
3021       False
3022        True

[3023 rows x 1 columns]
Linear equation: y= -0.02 x+ 0.06


C:\Users\476275\AppData\Local\Temp\ipykernel_12964\2226201627.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  coef = round(float(model.coef_), 2)
C:\Users\476275\AppData\Local\Temp\ipykernel_12964\2226201627.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  intercept = round(float(model.intercept_), 2)


In [42]:
#Location